## LOAD DATA

In [1]:
import collections
import pathlib
import re
import string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset/tf/main_lines_verses.csv", encoding= 'unicode_escape')

In [3]:
from sklearn.utils import shuffle
df = shuffle(df)

In [4]:
df.head()

,artist_name,line
52184,Eminem,"Though what you sacrifice barely is half, neve..."
146363,Gunna,"They flexed on me with rolls, shit don't compa..."
107826,Logic,"I got four cards and they all black, got four ..."
157376,Napalm,She was pissed and I ended up seein' my physician
97785,Kendrick Lamar,Taylor made a career out of music from writing...


In [5]:
df['artist_name'].value_counts().sort_index()

 2Pac & Tyrone Wrice     5
 Drake                  22
 Goldie Loc             79
 King Gordy             32
 Method Man             17
                        ..
iNTeLL                  16
mark curry              18
p. diddy over Hook       8
stic.man, dead prez      9
will.i.am               13
Name: artist_name, Length: 1262, dtype: int64

In [6]:
# df.replace("Jay-Z", "JAY-Z", inplace=True)
# df.replace("KRS-One", "KRS-ONE", inplace=True)
# df.replace("LL Cool J", "L.L. Cool J", inplace=True)
# df.replace("Royce da 5'9", "Royce Da 5'9", inplace=True)
df.replace("Notorious B.I.G.", "The Notorious B.I.G.", inplace=True)
df = df.groupby('artist_name').filter(lambda x : len(x)>1000)
df = df[df['line'].apply(lambda x: len(x.split(" ")) > 3)]

In [7]:
list = ['The Notorious B.I.G.', 'Ice Cube', 'Nas', '2Pac', 'Method Man', 'Eminem',
        'Snoop Dogg', 'DMX', 'Dr. Dre', 'GZA', 'The Notorious B.I.G.', 'RZA']
df = df[df['artist_name'].isin(list)]

In [8]:
df['artist_name'].value_counts()

Eminem                  8377
Ice Cube                4381
Nas                     3821
2Pac                    3775
Method Man              3331
The Notorious B.I.G.    2695
Snoop Dogg              2626
DMX                     2562
Dr. Dre                 2235
GZA                     2033
RZA                     1762
Name: artist_name, dtype: int64

In [9]:
lyrics_features = df['line']
lyrics_labels = df.pop('artist_name')

In [10]:
lyrics_labels

52184         Eminem
70146     Method Man
76335       Ice Cube
34093            DMX
81164         Eminem
             ...    
24910         Eminem
193116    Method Man
47997         Eminem
137243    Method Man
49209         Eminem
Name: artist_name, Length: 37598, dtype: object

In [11]:
set(lyrics_labels)

{'2Pac',
 'DMX',
 'Dr. Dre',
 'Eminem',
 'GZA',
 'Ice Cube',
 'Method Man',
 'Nas',
 'RZA',
 'Snoop Dogg',
 'The Notorious B.I.G.'}

In [12]:
len(set(lyrics_labels))

11

In [13]:
lyrics_labels = lyrics_labels.astype("category")

In [14]:
artist_ids = dict(enumerate(lyrics_labels.cat.categories))
artist_ids

{0: '2Pac',
 1: 'DMX',
 2: 'Dr. Dre',
 3: 'Eminem',
 4: 'GZA',
 5: 'Ice Cube',
 6: 'Method Man',
 7: 'Nas',
 8: 'RZA',
 9: 'Snoop Dogg',
 10: 'The Notorious B.I.G.'}

In [15]:
lyrics_labels = lyrics_labels.cat.codes

In [16]:
lyrics_labels

52184     3
70146     6
76335     5
34093     1
81164     3
         ..
24910     3
193116    6
47997     3
137243    6
49209     3
Length: 37598, dtype: int8

In [17]:
len(set(lyrics_labels))

11

In [18]:
lyrics_ds = tf.data.Dataset.from_tensor_slices((lyrics_features, lyrics_labels))

In [19]:
BUFFER_SIZE = 38000
BATCH_SIZE = 64
VALIDATION_SIZE = 3000

In [20]:
all_labeled_data = lyrics_ds.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

for text, label in all_labeled_data.take(10):
  print("Line: ", text.numpy())
  print("Artist:", label.numpy())

Line:  b'You are your car, what could represent me?'
Artist: 7
Line:  b"Yours is shabby and scab, while mine's glistenin'"
Artist: 7
Line:  b"Some niggas'll get money and pay niggas to back em'"
Artist: 2
Line:  b'Over girls like you with the BIG ol butts'
Artist: 5
Line:  b'Have the biggest dick, but when your shell get hit'
Artist: 10
Line:  b"Slow music, H-Town, no that's down low"
Artist: 7
Line:  b'All I did was give you a style for you to run with'
Artist: 7
Line:  b"They want heat, I'll give it to them burnt and crispy"
Artist: 4
Line:  b"Livin' underage, but he'll blaze on your bitch-ass"
Artist: 0
Line:  b'And up in yo bitch, is where ya might find me'
Artist: 9


In [21]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

### PROCESSING DATA FOR CNN CLASSIFICATION

In [21]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [22]:
def tokenize(text, unused_label):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [23]:
tokenized_ds = all_labeled_data.map(tokenize)

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


In [24]:
for text_batch in tokenized_ds.take(5):
  print("Tokens: ", text_batch.numpy())

Tokens:  [b'plenty' b'a' b'times' b'a' b'nigga' b'slipped' b'and' b'fell']
Tokens:  [b'talk' b'to' b'you' b'for' b'a' b'minute' b'then' b'my' b'dick' b"'"
 b's' b'in' b'you']
Tokens:  [b'and' b'that' b"'" b's' b'how' b'you' b'get' b'it' b',20' b'years' b'in'
 b'a' b'row']
Tokens:  [b'anybody' b'in' b'your' b'crew' b'when' b'i']
Tokens:  [b'he' b'inhaled' b'so' b'deep' b',' b'shut' b'his' b'eyes' b'like' b'he'
 b'was' b'sleep']


In [26]:
VOCAB_SIZE = 10000

In [27]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1

vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries:", vocab[:5])

Vocab size:  10000
First five vocab entries: [b"'", b',', b'the', b'i', b'a']


In [28]:
keys = vocab
values = range(2, len(vocab) + 2)  # reserve 0 for padding, 1 for OOV

init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

In [29]:
def preprocess_text(text, label):
  standardized = tf_text.case_fold_utf8(text)
  tokenized = tokenizer.tokenize(standardized)
  vectorized = vocab_table.lookup(tokenized)
  return vectorized, label

In [30]:
example_text, example_label = next(iter(all_labeled_data))
print("Sentence: ", example_text.numpy())
vectorized_text, example_label = preprocess_text(example_text, example_label)
print("Vectorized sentence: ", vectorized_text.numpy())

Sentence:  b'Plenty a times a nigga slipped and fell'
Vectorized sentence:  [1570    6  417    6   50 2093    9  773]


In [31]:
all_encoded_data = all_labeled_data.map(preprocess_text)

In [32]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

In [33]:
train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

In [34]:
sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
print("First text example: ", sample_text[0])
print("First label example: ", sample_labels[0])

Text batch shape:  (64, 19)
Label batch shape:  (64,)
First text example:  tf.Tensor(
[1570    6  417    6   50 2093    9  773    0    0    0    0    0    0
    0    0    0    0    0], shape=(19,), dtype=int64)
First label example:  tf.Tensor(1, shape=(), dtype=int8)


In [35]:
vocab_size += 2

In [36]:
train_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

## TRAIN MODELS

### CLASSIFIER 1

In [37]:
def create_model(vocab_size, num_labels):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, 64, mask_zero=True),
        layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
        layers.GlobalMaxPooling1D(),
        layers.Dense(num_labels)
    ])
    return model

In [38]:
model = create_model(vocab_size=vocab_size, num_labels=11)
model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
history = model.fit(train_data, validation_data=validation_data, epochs=20)

Epoch 1/20
541/541 [==============================] - 52s 33ms/step - loss: 2.2648 - accuracy: 0.2260 - val_loss: 2.0192 - val_accuracy: 0.3087
Epoch 2/20
541/541 [==============================] - 11s 20ms/step - loss: 1.8986 - accuracy: 0.3679 - val_loss: 1.8757 - val_accuracy: 0.3720
Epoch 3/20
541/541 [==============================] - 11s 20ms/step - loss: 1.5203 - accuracy: 0.5114 - val_loss: 1.8685 - val_accuracy: 0.3977
Epoch 4/20
541/541 [==============================] - 11s 20ms/step - loss: 1.1850 - accuracy: 0.6291 - val_loss: 1.9533 - val_accuracy: 0.3983
Epoch 5/20
541/541 [==============================] - 11s 20ms/step - loss: 0.9027 - accuracy: 0.7252 - val_loss: 2.1017 - val_accuracy: 0.3957
Epoch 6/20
541/541 [==============================] - 12s 22ms/step - loss: 0.6834 - accuracy: 0.8016 - val_loss: 2.2861 - val_accuracy: 0.3887
Epoch 7/20
541/541 [==============================] - 11s 20ms/step - loss: 0.5180 - accuracy: 0.8578 - val_loss: 2.4933 - val_accuracy:

In [39]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

47/47 [==============================] - 2s 3ms/step - loss: 5.4669 - accuracy: 0.3633
Loss:  5.4669389724731445
Accuracy: 36.33%


In [40]:
model.save('lyrics_classificer_1')

INFO:tensorflow:Assets written to: lyrics_classificer_1\assets


### CLASSIFIER 1.1

In [41]:
def create_model(vocab_size, num_labels):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, 16),
        layers.Dropout(0.2),
        layers.GlobalAveragePooling1D(),
        layers.Dropout(0.2),
        layers.Dense(num_labels)])
    return model

In [42]:
model = create_model(vocab_size=vocab_size, num_labels=11)
model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
history = model.fit(train_data, validation_data=validation_data, epochs=20)

Epoch 1/20
541/541 [==============================] - 6s 9ms/step - loss: 2.3385 - accuracy: 0.2106 - val_loss: 2.2655 - val_accuracy: 0.2147
Epoch 2/20
541/541 [==============================] - 4s 8ms/step - loss: 2.2423 - accuracy: 0.2286 - val_loss: 2.2174 - val_accuracy: 0.2257
Epoch 3/20
541/541 [==============================] - 4s 8ms/step - loss: 2.1892 - accuracy: 0.2461 - val_loss: 2.1632 - val_accuracy: 0.2493
Epoch 4/20
541/541 [==============================] - 4s 8ms/step - loss: 2.1293 - accuracy: 0.2703 - val_loss: 2.1089 - val_accuracy: 0.2817
Epoch 5/20
541/541 [==============================] - 4s 8ms/step - loss: 2.0703 - accuracy: 0.2996 - val_loss: 2.0608 - val_accuracy: 0.2977
Epoch 6/20
541/541 [==============================] - 4s 8ms/step - loss: 2.0156 - accuracy: 0.3266 - val_loss: 2.0187 - val_accuracy: 0.3213
Epoch 7/20
541/541 [==============================] - 4s 8ms/step - loss: 1.9603 - accuracy: 0.3485 - val_loss: 1.9810 - val_accuracy: 0.3380
Epoch 

In [43]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

47/47 [==============================] - 2s 2ms/step - loss: 1.7608 - accuracy: 0.4230
Loss:  1.7608269453048706
Accuracy: 42.30%


In [44]:
model.save('lyrics_classificer_1_1')

INFO:tensorflow:Assets written to: lyrics_classificer_1_1\assets


### CLASSIFIER 1.2

In [45]:
def create_model(vocab_size, num_labels):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, 64, mask_zero=True),
        layers.Dropout(0.2),
        layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
        layers.Dropout(0.2),
        layers.GlobalMaxPooling1D(),
        layers.Dropout(0.2),
        layers.Dense(num_labels)
    ])
    return model

In [46]:
model = create_model(vocab_size=vocab_size, num_labels=11)
model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
history = model.fit(train_data, validation_data=validation_data, epochs=20)

Epoch 1/20
541/541 [==============================] - 13s 22ms/step - loss: 2.2856 - accuracy: 0.2143 - val_loss: 2.1039 - val_accuracy: 0.2833
Epoch 2/20
541/541 [==============================] - 14s 25ms/step - loss: 2.0216 - accuracy: 0.3177 - val_loss: 1.9324 - val_accuracy: 0.3600
Epoch 3/20
541/541 [==============================] - 13s 24ms/step - loss: 1.7406 - accuracy: 0.4265 - val_loss: 1.8579 - val_accuracy: 0.3820
Epoch 4/20
541/541 [==============================] - 13s 24ms/step - loss: 1.5106 - accuracy: 0.5049 - val_loss: 1.8520 - val_accuracy: 0.3903
Epoch 5/20
541/541 [==============================] - 12s 23ms/step - loss: 1.3182 - accuracy: 0.5697 - val_loss: 1.8784 - val_accuracy: 0.3950
Epoch 6/20
541/541 [==============================] - 11s 21ms/step - loss: 1.1586 - accuracy: 0.6243 - val_loss: 1.9453 - val_accuracy: 0.3907
Epoch 7/20
541/541 [==============================] - 11s 21ms/step - loss: 1.0364 - accuracy: 0.6648 - val_loss: 2.0219 - val_accuracy:

In [47]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

47/47 [==============================] - 2s 4ms/step - loss: 3.2865 - accuracy: 0.3783
Loss:  3.2865211963653564
Accuracy: 37.83%


In [48]:
model.save('lyrics_classificer_1_2')

INFO:tensorflow:Assets written to: lyrics_classificer_1_2\assets
